In [1]:
import os 
import json
import pandas as pd
import traceback

In [2]:
%pip install langchain_openai

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_openai import ChatOpenAI

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
KEY = os.getenv("OpenAI_API_Key")

In [6]:
from pydantic import SecretStr

llm = ChatOpenAI(api_key=SecretStr(KEY) if KEY is not None else None, model="gpt-3.5-turbo", temperature=0.5)